# **CSC14120 - PARALLEL PROGRAMMING REPORT**





## 1. Thông tin Sinh viên
|MSSV  | Họ và tên | Email |
|:---------:|:--------------------|:-----------------------------|
|20127206   |Vũ Đình Duy Khánh    |vddkhanh20@clc.fitus.edu.vn   |
|20127478   |Huỳnh Tuấn Duy       |htduy20@clc.fitus.edu.vn      |

## 2. Giới thiệu
Ứng dụng của chúng tôi tập trung vào tối ưu hóa việc thực hiện quá trình truyền tiến của một lớp convolutional bằng CUDA. Được phát triển trên cơ sở Mini-DNN và sử dụng tập dữ liệu Fashion MNIST, đồ án này nhằm cải thiện hiệu suất inference cho các lớp Convolutional 1 và Convolutional 3 trong mô hình LeNet-5.

## 3. Động lực thực hiện
Xây dựng một lớp convolutional hiệu suất cao bằng CUDA trong đồ án này không chỉ nâng cao kỹ năng lập trình mà còn đưa ra giải pháp ứng dụng trong thế giới thực, tăng cường khả năng xử lý ảnh và thông tin, làm nền tảng cho nhiều ứng dụng trong lĩnh vực trí tuệ nhân tạo

## 4. Liên kết
- Video Demo : https://www.youtube.com/watch?v=ZtMTfWKptTM
- Team plan : [Team plan và work distribution](PlanAndWorkDistribution.ipynb)

## 5. Tổng quan đóng góp
|MSSV  | Họ và tên | Tỷ lệ đóng góp |
|:---------:|:--------------------|:-----------------------------:|
|20127206   |Vũ Đình Duy Khánh    | 50%  |
|20127478   |Huỳnh Tuấn Duy       |50%      |

## 6. Cài đặt trên Google Colab

In [ ]:
!git clone https://github.com/tuanduy18012002/LeNet-5-using-Cuda-C-Programming
%cd /content/LeNet-5-using-Cuda-C-Programming
!mkdir build
%cd build

In [ ]:
!cmake ../CMakeLists.txt

In [ ]:
!make

## 7. Thực thi tuần tự (CPU)

In [ ]:
!./cpu_main

<-------------HOST------------->
mnist train number: 60000
mnist test number: 10000
<------------------------------>
CPU - Forward Convolution - Layer 1 - Time: 36127.070312 ms
CPU - Forward Convolution - Layer 3 - Time: 30667.855469 ms
test accuracy: 0.8779
<------------------------------>


### Chi tiết
* Flow thiết kế:
  - Khởi tạo biến và thời gian, resize ma trận (top), và chuẩn bị một vector (data_cols) để lưu trữ ma trận dạng cột từ quá trình im2col.
  - Dùng vòng lặp để xử lý từng mẫu đầu vào. Với mỗi mẫu, áp dụng im2col để tạo ma trận dạng cột, sau đó thực hiện convolution bằng cách nhân với trọng số và cộng thêm bias.
  - Mỗi mẫu đầu vào được chuyển qua hàm im2col, tạo ra ma trận dạng cột và lưu vào (data_cols).
  -  Ma trận kết quả (result) được tính thông qua phép nhân ma trận dạng cột với ma trận trọng số (weight), sau đó cộng thêm bias.
* Tốc độ thực thi
  - Thời gian chạy của Convolution Kernel - Layer 1 khoảng 36s.
  - Thời gian chạy của Convolution Kernel - Layer 3 khoảng 30s.
* Độ chính xác: 87.8%

## 8. Thực thi song song hoá (GPU)

In [6]:
!./gpu_main

<------------DEVICE------------>
mnist train number: 60000
mnist test number: 10000
<---------DEVICE-INFO---------->
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************
<----------VERSION--1---------->
GPU - Forward Convolution - Layer 1 - Time: 120.532867 ms
GPU - Forward Convolution - Layer 3 - Time: 64.140129 ms
test accuracy version 1: 0.8779
<----------VERSION--2---------->
GPU - Forward Convolution - Layer 1 - Time: 140.262756 ms
GPU - Forward Convolution - Layer 3 - Time: 136.350586 ms
test accuracy version 2: 0.8775
<----------VERSION--3---------->
GPU - Forward Convolution - Layer 1 - Time: 128.134201 ms
GPU - Forward Convolution - Layer 3 - Time: 55.752159 ms
test accuracy version 3: 0.8775
<------------------------------>


### Chi tiết
#### a. Version 1
* Phân tích:
  - Trong Conv.forward trong CPU có một vòng for từ 0 --> n-sample. Vì vậy, dùng song song để tối ưu thời gian chạy vòng for này.
* Flow thiết kế:
  - Sử dụng một phương pháp đơn giản với các vòng lặp lồng nhau để duyệt qua các kênh đầu vào, hàng kernel và cột kernel thực hiện phép tích chập trên GPU cho một đầu vào, trọng số và tạo ra đầu ra một cách song song.
  - Phương pháp tích chập: Sử dụng vòng lặp lồng nhau để duyệt qua các kênh đầu vào, hàng kernel và cột kernel.
  - Sử dụng nested loops để duyệt qua từng phần tử trong tensor đầu vào và tensor trọng số, thực hiện phép nhân và cộng để tính tổng trọng số của đầu vào với kernel tại từng vị trí.
  - Kiểm tra giới hạn trước khi ghi vào đầu ra để đảm bảo tính hợp lý và tránh ghi vào kích thước bộ nhớ không hợp lý.
  - Phép cộng atomic: Sử dụng atomicAdd để đảm bảo tính đồng nhất khi cập nhật giá trị đầu ra.
* Đánh giá
  - Tốc độ thực thi:
    + Convolution kernel 1: khoảng 120ms
    + Convolution kernel 2: khoảng 64ms
    + Giảm khoảng 475 lần thời gian chạy trên CPU
  - Độ chính xác: 87.8%

#### b. Version 2
* Phân tích
  - Dùng shared memory, fp16 và nStream để tối ưu hoá cho version 1.
* Flow Thiết kế
  - Sử dụng shared memory để lưu trữ trọng số
  - Sử dụng FP16 (dấu chấm động bán chính xác half và half2) để giảm yêu cầu bộ nhớ và tăng tốc độ tính toán.
  - Hỗ trợ nhiều streams để xử lý các batch không đồng bộ, giúp tận dụng tối đa khả năng chịu tải của GPU.
* Đánh giá
  - Tốc độ thực thi:
    + Convolution kernel - Layer 1: khoảng 140ms
    + Convolution kernel - Layer 3: khoảng 136ms
    + Giảm khoảng 250 lần thời gian chạy trên CPU
  - Độ chính xác: 87.8%

#### c. Version 3
* Phân tích
  - Dùng thêm constant memory, kiến trúc tiling chia nhỏ phép tính cho version 2.
* Thiết kế
  - Phép tích chập được thực hiện theo cách chia thành từng ô, với mỗi khối xử lý một ô của bản đồ đặc trưng đầu ra.
  - Sử dụng shared memory để lưu trữ trọng số
  - Sử dụng constant memory để lưu trữ trọng số, giảm độ trễ khi truy cập dữ liệu trọng số.
  - Sử dụng FP16 (dấu chấm động bán chính xác half và half2) để giảm yêu cầu bộ nhớ và tăng tốc độ tính toán.
  - Sử dụng kiến trúc tiling để chia nhỏ phép tính toán thành các phần nhỏ hơn, giúp tối ưu hóa hiệu suất trong quá trình xử lý.
* Đánh giá
  - Tốc độ thực thi:
    + Convolution kernel - Layer 1: khoảng 128ms
    + Convolution kernel - Layer 3 : khoảng 55ms
    + Giảm khoảng 500 lần thời gian chạy trên CPU
  - Độ chính xác: 87.8%

## 9. Tự đánh giá đồ án (9.5/10)
#### a. Đã làm được:
* Xây dựng framework hoàn chỉnh song song hoá và tích hợp.
* Xây dựng tuần tự (CPU).
* Xây dựng song song hoá cơn bản.
* Tối ưu hoá cho phiên bản song song hoá cơ bản với nhiều version.

#### b. Chưa làm được:
* Quét các tham số khác nhau để tìm giá trị tốt nhất (block sizes, amount of
thread coarsening).

#### c. Tiếp theo:
* Quét các tham số khác nhau để tìm giá trị tốt nhất.
* Tối ưu hoá thêm nếu có thể.

#### d. Khó khăn:
* Quản lý bộ nhớ và hiệu suất ứng dụng.
* Debugging và Profiling.

#### e. Giải quyết:
* Quản lý bộ nhớ và hiệu suất ứng dụng: chia nhỏ các tác vụ lớn thành các tác vụ nhỏ hơn.
* Debugging và Profiling: Đặt Debug Log và sử dụng các công cụ như NVIDA Nsight...